# Weigthed proximity analysis

Determine the amenities and servicies oppotunites with a weigthed value. The weigthed value is obtained dividing the amount of opportunities within 15min by the total amaount of opportunities for a given category.

## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/edgaregurrola/Repos/observatorio-ciudades/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## Download data

Proximity hexagons

In [2]:
hex_prox = aup.gdf_from_db('volvo_proxanalysis_2024_mza_hex9', 'projects_research')
print(hex_prox.shape)
hex_prox.head(2)

(5531, 50)


,hex_id,res,max_escuelas,min_escuelas,time_guarderías,time_preescolar,time_primaria,time_secundaria,max_salud,min_salud,...,cultural_15min,cines_15min,museos_15min,bibliotecas_15min,financiero_15min,bancos_15min,pobtot,dens_pob_ha,geometry,imn_2020
0,8949aa252bbffff,9,65.700136,5.424581,65.700136,18.338011,15.153014,5.424581,65.126155,13.229981,...,0.0,0.0,0.0,0.0,0.333333,0.333333,895.0,72.377587,"POLYGON ((-103.40480 20.46253, -103.40358 20.4...",0.931389
1,8949ab5943bffff,9,38.509210,27.956565,29.104480,28.120892,27.956565,38.509210,18.872854,17.391323,...,0.0,0.0,0.0,0.0,6.055556,6.055556,19.0,1.536037,"POLYGON ((-103.40982 20.60369, -103.40860 20.6...",0.977710


Amenities and services

In [12]:
# ---------------------------- BASE DATA REQUIRED ----------------------------
# Area of interest (city)
metro_schema = 'metropolis'
metro_table = 'metro_gdf_2020' #'metro_gdf_2015' or 'metro_gdf_2020'
# Network data (nodes and edges table for distance analysis,
# also used to generate the network G with which the nearest OSMID is assigned to each poi)
network_schema = 'osmnx'
nodes_table = 'nodes' #'nodes' or 'nodes_23_point'
edges_table = 'edges_speed' ################################################################################################## PENDIENTE
# Points of interest - DENUE
denue_schema = 'denue'
denue_table = 'denue_23_point' #'denue_2020' or 'denue_23_point'
# Points of interest - CLUES
clues_schema = 'denue'
clues_table = 'clues_23_point' #'clues' or 'clues_23_point'
# Points of interest - SIP
sip_schema = 'denue'
sip_table = 'sip_23_point' #'sip_2020' or 'sip_23_point'
# Points of interest - Espacio publico (Parques) ### Exclusive project Volvo
parques_schema = 'espacios_publicos' 
parques_table = 'ep_amg'
# Hexgrid
hex_schema = 'hexgrid'
# Population data
pop_schema = 'censo'
pop_table = 'hex_censo_mza_2020_res9'

In [4]:
gdf = hex_prox.to_crs("EPSG:6372")
gdf = gdf.buffer(500).reset_index().rename(columns={0: "geometry"})
gdf = gdf.set_geometry("geometry")
gdf = gdf.to_crs("EPSG:4326")
poly_wkt = gdf.dissolve().geometry.to_wkt()[0]

In [23]:
parameters = {'Escuelas':{'Guarderías':{'denue_guarderias':[624411, 624412]},
                          'Preescolar':{'denue_preescolar':[611111, 611112]},
                          'Primaria':{'denue_primaria':[611121, 611122]},
                          'Secundaria':{'denue_secundaria':[611131, 611132]}
                         },
              'Salud':{'Primer nivel':{'clues_primer_nivel':[8610]},
                       'Farmacias':{'denue_farmacias':[474111,464112]} ########## Volvo: 464111 Farmacias sin minisúper, 464112 Farmacias con minisúper
                       },
              'Parques':{'Area verde':{'odc_parques':[9321]} ########## Volvo: AGREGAR DE BD, 
                                                          ########## Se le asignó el code 9321, correspondiente a 'Activities of amusement parks and theme parks'
                                                          ########## Fuente del code elegido: International Standard Industrial Classification of all Economic Activities, Rev.4
                                                          ########## https://unstats.un.org/unsd/publication/seriesm/seriesm_4rev4e.pdf
                        },
              'Equipamiento deportivo':{'Canchas':{'sip_cancha':[93110]},
                                        'Unidad deportiva':{'sip_unidad_deportiva':[93111]}
                                       },
              'Cultural':{'Cines':{'denue_cines':[512130]},
                          'Museos':{'denue_museos':[712111, 712112]},
                          'Bibliotecas':{'denue_bibliotecas':[519121,519122]}
                         },
              'Financiero':{'Bancos':{'denue_bancos':[522110]} #################################################### Volvo: 522110 Banca Múltiple (Bancos y cajeros)
                           }
             }

In [24]:
param = parameters
param['Escuelas']['Preescolar'].append(100)

AttributeError: 'dict' object has no attribute 'append'

In [21]:
param['Escuelas']['Preescolar']

100

In [10]:
####################################################################################################################################
# ADAPTATION
# Simplified version, does not filter centro cultural nor dif because it is not used in project Volvo.
####################################################################################################################################
def get_denue_pois(denue_schema,denue_table,poly_wkt,code):
    # This function downloads the codigo_act denue poi requested for the analysis.

    # Download denue pois
    query = f"SELECT * FROM {denue_schema}.{denue_table} WHERE (ST_Intersects(geometry, \'SRID=4326;{poly_wkt}\')) AND (\"codigo_act\" = \'{code}\')"
    code_pois = aup.gdf_from_query(query, geometry_col='geometry')

    # Format denue pois
    code_pois = code_pois[['codigo_act', 'geometry']]
    code_pois = code_pois.rename(columns={'codigo_act':'code'})
    code_pois['code'] = code_pois['code'].astype('int64')

    return code_pois

In [11]:
####################################################################################################################################
# NEW
# Project Volvo includes 'parques', which are in OdC's DB
####################################################################################################################################
def get_parques_pois(parques_schema,parques_table,code):
    # This function creates parques points of interest out of vertices of parques found in db (schema>table)

    # Download parques pois
    query = f"SELECT * FROM {parques_schema}.{parques_table}"
    gdf = aup.gdf_from_query(query, geometry_col='geometry')
    gdf = gdf.to_crs("EPSG:4326")

    # Get vertices coordinates
    gdf_coords = gdf.geometry.get_coordinates()

    # Merge back with gdf containing data
    gdf_coords_data = pd.merge(gdf_coords,gdf,left_index=True,right_index=True)

    # Drop poly geometry 
    df_coords_data = gdf_coords_data.drop(columns=['geometry'])
    # Set points geometry
    gdf_2 = gpd.GeoDataFrame(df_coords_data, 
                             geometry=gpd.points_from_xy(df_coords_data.x, df_coords_data.y),
                             crs='EPSG:4326')
    # Format
    gdf_2.drop(columns=['x','y'],inplace=True)
    gdf_2.reset_index(inplace=True)
    gdf_2.rename(columns={'index':'polygon_id'},inplace=True)

    # Project Volvo filter
    # Tipos
    tipos = ['Parque','Espacio verde vecinal', 'Plaza', 'Área natural', 'Área natural protegida']
    gdf_f1 = gdf_2.loc[gdf_2.Tipo.isin(tipos)]
    # Usos
    gdf_f2 = gdf_f1.loc[gdf_f1.Uso == 'Uso recreativo']

    # Proxanalysis filter
    #code_pois = gdf_f2.set_crs("EPSG:4326")
    gdf_f2['code'] = code
    code_pois = gdf_f2[['polygon_id','code','geometry']]
    code_pois['code'] = code_pois['code'].astype('int64')

    code_pois = code_pois.drop_duplicates()

    return code_pois

Download CLUES and SIP

In [14]:
# 1.2 --------------- DOWNLOAD POINTS OF INTEREST (clues and sip pois, not denue)
# ------------------- This step downloads SIP and CLUES points of interest (denue pois are downloaded later.)

aoi = hex_prox[['hex_id','geometry']].copy()
city = 'Guadalajara'

sip_clues_gdf = gpd.GeoDataFrame()

# CLUES (Salud)
aup.log(f"--- Downloading CLUES pois for {city}.")
# Download
clues_gdf = aup.gdf_from_polygon(aoi, clues_schema, clues_table, geom_col="geometry")
# Filter
clues_pois = clues_gdf.loc[clues_gdf['nivel_atencion'] == 'PRIMER NIVEL'].copy()
del clues_gdf
# Format
clues_pois.loc[:,'code'] = 8610
clues_pois = clues_pois[['code','geometry']]
# Save to pois_tmp
sip_clues_gdf = pd.concat([sip_clues_gdf,clues_pois])
del clues_pois

# SIP (Marco geoestadistico)
# Download
sip_gdf = aup.gdf_from_polygon(aoi, sip_schema, sip_table, geom_col="geometry")
sip_amenities = {'GEOGRAFICO':['Mercado','Plaza'], 
                 'TIPO':['Cancha','Unidad Deportiva','Áreas Verdes','Jardín','Parque']}
# Filter - SIP pois of interest
sip_amenities_codes = {'Mercado':4721, #sip_mercado
                       'Cancha':93110, #sip_cancha
                       'Unidad Deportiva':93111, #sip_unidad_deportiva 
                       'Áreas Verdes':9321, #sip_espacio_publico 
                       'Jardín':9321, #sip_espacio_publico
                       'Parque':9321, #sip_espacio_publico
                       'Plaza':9321 #sip_espacio_publico
                        }
# Filter - Iterate over sip_amenities and filter sip gdf
sip_pois = gpd.GeoDataFrame()
for col in sip_amenities:
    for amenity in sip_amenities[col]:
        sip_tmp = sip_gdf.loc[sip_gdf[col] == amenity]
        sip_tmp.loc[:,'code'] = sip_amenities_codes[amenity]
        sip_pois = pd.concat([sip_pois,sip_tmp])
del sip_gdf
# Format
sip_pois = sip_pois[['code','geometry']]
# Save to pois_tmp
sip_clues_gdf = pd.concat([sip_clues_gdf,sip_pois])
del sip_pois

/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/edgaregurrola/anaconda3/envs/odc/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on 

In [26]:
####################################################################################################################################
# ADAPTATION
# Added option for source being odc (In order skip it since for parks it will be slightly different).
# Removed 'version' from get_denue_pois
# Output renamed (nodes_analysis_1), will be concatenated with nodes_analysis_2 (Greenspace analysis)
####################################################################################################################################

# 1.3a --------------- ANALYSE POINTS OF INTEREST (If denue, downloads)
# ------------------- This step analysis times (and count of pois at given time proximity if requested) using function aup.pois_time.

poly_wkt = aoi.dissolve().geometry.to_wkt()[0]

i = 0
analysis_cols = []

for eje in parameters.keys():
    for amenity in parameters[eje]:
        for source in parameters[eje][amenity]:
            if source[0] == 'o':
                print(f"---SKIPPING {source}.")
                continue
            
            print(f"""---Analysing source {source}.""")
            
            analysis_cols.append(source)
            
            # ANALYSIS - Select source points of interest
            source_pois = gpd.GeoDataFrame()
            for code in parameters[eje][amenity][source]:
                #If source is denue:
                if source[0] == 'd':
                    print(f'--- Downloading denue source pois code {code} from db.')
                    code_pois = get_denue_pois(denue_schema,denue_table,poly_wkt,code)
                #If source is clues or sip:
                elif source[0] == 'c' or source[0] == 's':
                    print(f'--- Getting clues/sip source pois code {code} from previously downloaded.')
                    code_pois = sip_clues_gdf.loc[sip_clues_gdf['code'] == code]
                else:
                    print(f'--- Error, check parameters dicctionary.')
                    print(f'--- Sources must start with denue_, clues_, odc_ or sip_.')
                    intended_crash
                    
                source_pois = pd.concat([source_pois,code_pois])

            print(f"--- {source_pois.shape[0]} {source} pois. Analysing source pois proximity to nodes.")
            

---Analysing source denue_guarderias.
--- Downloading denue source pois code 624411 from db.
--- Downloading denue source pois code 624412 from db.
--- 426 denue_guarderias pois. Analysing source pois proximity to nodes.
---Analysing source denue_preescolar.
--- Downloading denue source pois code 611111 from db.
--- Downloading denue source pois code 611112 from db.
--- 1667 denue_preescolar pois. Analysing source pois proximity to nodes.
---Analysing source denue_primaria.
--- Downloading denue source pois code 611121 from db.
--- Downloading denue source pois code 611122 from db.
--- 1475 denue_primaria pois. Analysing source pois proximity to nodes.
---Analysing source denue_secundaria.
--- Downloading denue source pois code 611131 from db.
--- Downloading denue source pois code 611132 from db.
--- 241 denue_secundaria pois. Analysing source pois proximity to nodes.
---Analysing source clues_primer_nivel.
--- Getting clues/sip source pois code 8610 from previously downloaded.
--- 69

In [27]:
source_pois

,code,geometry
0,522110,POINT (-103.37861 20.73742)
1,522110,POINT (-103.37203 20.74015)
2,522110,POINT (-103.44355 20.78659)
3,522110,POINT (-103.40332 20.74113)
4,522110,POINT (-103.40539 20.73944)
...,...,...
2736,522110,POINT (-103.42517 20.71812)
2737,522110,POINT (-103.41853 20.73055)
2738,522110,POINT (-103.35142 20.63716)
2739,522110,POINT (-103.33194 20.64384)
